In [164]:
import re
import os
from os import path
import types
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize
import types

def chapter_splitter(file, pattern): #THE FUNCTION IS OPTIONAL. TO DELIGATE FUNCTIONALITY: 1) PATH CHECKER/MAKER
    #2) FILE SPLITTER 3) CHAPTER MAKER
    """The function splits up .txt utf-8 file into separate files by chapters and saves them 
    in corpus_data directory. To call the function, put a file-path and re-pattern as arguments. 
    Returns a generator with file-paths of the chapters for processing by other functions"""
    current_path = os.getcwd()
    current_path = current_path+'\\corpus_data\\'
    if path.exists(current_path) == True:
        pass
    elif path.exists(current_path) == False:
        os.mkdir('corpus_data')
    
    pattern = re.compile(pattern)

    with open(file, 'r', encoding='utf-8') as fp:
        doc = fp.readlines()
        n = 0 #Chapter number counter
        chapter_path = []
        for line in doc:
            if n == 0:
                output = open('corpus_data\intro.txt'.format(n), 'w', encoding='utf-8') #TO FIX INTRO!!
                output.write(line)
            matches = pattern.finditer(line)

            for match in matches:
                n += 1
                output = open('corpus_data\chapter_{}.txt'.format(n), 'w', encoding='utf-8')
                chapter_path += ['corpus_data\\chapter_{}.txt'.format(n)]
            output.write(line)

        output.close()

        for chapter in chapter_path: yield chapter #Generator with filepaths
    
ch = chapter_splitter('Fuji_lemmatizedUTF.txt', r'\d\.\d.?\s')



def sent_splitter(file_path_input): #THE FUNCTION IS NECESSARY, IT SHOULD BE ABLE TO TAKE CHAPTER_SPLITTER FUNC
    #TO MAKE TWO FUNCTIONS AND DELEGATE PART OF FUNCTIONALITY: 1) SORTING; 2) SPLITTING
    """Sent_splitter can be used with chapter_splitter or without it.
    Splits text on sentences and saves them into a list. Takes on filepath as an argument (string)
    or multiple filepaths packed in a list of strings or a generator. Removes punctuation"""
    
    if isinstance(file_path_input, str): 
        print('The file type is a string. A single file has passed.')
        with open(file_path_input, 'r', encoding='utf-8') as fp:
            fp = fp.read() #TO READLINES()
        return sent_tokenize(fp)

    
    elif isinstance(file_path_input, types.GeneratorType):      
        print('The file type is a generator. Multiple files have passed.')
        for file in file_path_input:
            with open(file, 'r', encoding='utf-8') as fp:
                fp = fp.read() #TO READLINES()
                return sent_tokenize(fp)

        
    elif isinstance(file_path_input, list):
        print('The file type is a list. Multiple files have passed.')
        for file in file_path_input:
            with open(file, 'r', encoding='utf-8') as fp:
                fp = fp.read() #TO READLINES()
                return sent_tokenize(fp)

    else:
        print('Unexpected error occured. The data type is a {}'.format(type(file_path_input)))

#sent = sent_splitter(ch)
chapter_gen = (sent_splitter(i) for i in ch)


def clean_sentences(sentences): #TO DELEGATE DATA_FRAME FUNCTION, TO GET RID OF COPIER
    puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';',
                ', ', '. ', '! ', '? ', ': ', '""', '— ', '« ', '» ', ') ', '(', '; ',
                ' — ', ', — ', '; ', ' (', ' «', '-', '), ', ': «', '», ', ']',
                '[', '] ', '–', '…'))
    stopwords = nltk.corpus.stopwords.words('russian')
    
    sentences = [(letter.replace('\n', ' ') for letter in sentence)
                        for sentence in sentences]
    sentences = [''.join(letter for letter in sentence if letter not in puncs)
                        for sentence in sentences]
    df = pd.DataFrame(sentences, columns=['text'])

    return df

#clean_sentences(sent)



In [163]:
ch = chapter_splitter('Fuji_lemmatizedUTF.txt', r'\d\.\d.?\s')
chapter_gen = (sent_splitter(i) for i in ch)

def clean_chapter_by_chapter(chapters):
    puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';',
            ', ', '. ', '! ', '? ', ': ', '""', '— ', '« ', '» ', ') ', '(', '; ',
            ' — ', ', — ', '; ', ' (', ' «', '-', '), ', ': «', '», ', ']',
            '[', '] ', '–', '…'))
    for chapter in chapters:
        for sentence in chapter:
            sentences = ((letter.replace('\n', ' ') for letter in sentence)
                        for sentence in chapter)
            sentences = (''.join(letter for letter in sentence if letter not in puncs)
                        for sentence in sentences)
    
        df = pd.DataFrame(sentences, columns=['text'])

        return df
    
#corpus = clean_chapter_by_chapter(chapter_gen)


In [226]:
ch = chapter_splitter('Fuji_lemmatizedUTF.txt', r'\d\.\d.?\s')
chapter_gen = (sent_splitter(i) for i in ch)

def clean_chapters(chapters): #DONE!!
    puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';',
                ', ', '. ', '! ', '? ', ': ', '""', '— ', '« ', '» ', ') ', '(', '; ',
                ' — ', ', — ', '; ', ' (', ' «', '-', '), ', ': «', '», ', ']',
                '[', '] ', '–', '…'))
    stopwords = nltk.corpus.stopwords.words('russian')
    
    chapters = ((sentence.replace('\n', ' ') for sentence in chapter)
                        for chapter in chapters)
    chapters = ((''.join(letter for letter in sentence if letter not in puncs)
                        for sentence in chapter)
                        for chapter in chapters)
    chapters = (' '.join(sentence for sentence in chapter) for chapter in chapters)
    df = pd.DataFrame(chapters, columns=['text'])

    return df

#corpus = clean_chapters(chapter_gen)
#corpus

In [224]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize

tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=corpus.text).toarray()

tfidf_docs = pd.DataFrame(tfidf_docs)
tfidf_docs = tfidf_docs - tfidf_docs.mean()
#tfidf_docs.shape

In [223]:
from sklearn.decomposition import PCA

pca = PCA(n_components=4)
pca = pca.fit(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
index = ['chapter{}'.format(i) for i in range(len(corpus))]
columns = ['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=index)
#pca_topic_vectors.round(2).head(106)

In [189]:
column_nums, terms = zip(*sorted(zip(tfidf.vocabulary_.values(), tfidf.vocabulary_.keys())))
#terms

In [222]:
weights = pd.DataFrame(pca.components_, columns=terms, index=['topic{}'.format(i) for i in range(4)])
pd.options.display.max_columns = 8
#weights.head(10).round(3)

In [220]:
pd.options.display.max_columns = 12
deals = weights['ящерица джан джана таня четвертый кларисса кокаин я будда'.split()].round(3)*100
#deals

In [219]:
#deals.T.sum()

In [218]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=4, n_iter=100)
svd_topic_vectors = svd.fit_transform(tfidf_docs.values)
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns, index=index)
#svd_topic_vectors.round(2).head(9)

In [200]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=corpus.text).toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(), counter.vocabulary_.keys())))
bow_docs.columns = terms

In [203]:
#corpus.loc[2].text

In [221]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA

ldia = LDiA(n_components=4, learning_method='batch')
ldia = ldia.fit(bow_docs)
#ldia.components_.shape

In [212]:
pd.set_option('display.width', 75)
components = pd.DataFrame(ldia.components_.T, index=terms, columns=columns)
#components.round(2).head(50)

In [217]:
#components.topic3.sort_values(ascending=False)[:60]